# Qwen2-Vision Fine-Tuning on Kaggle with Unsloth and Geo170K

This notebook demonstrates how to fine-tune a Qwen2-Vision model using the [Unsloth](https://github.com/unslothai/unsloth) library on Kaggle, following the G-LLaVA procedure and using the [Geo170K dataset](https://huggingface.co/datasets/Luckyjhg/Geo170K) from Hugging Face.

**References:**
- [G-LLaVA Fine-tuning Procedure](https://github.com/pipilurj/G-LLaVA)
- [Unsloth Docs](https://github.com/unslothai/unsloth)
- [Geo170K Dataset](https://huggingface.co/datasets/Luckyjhg/Geo170K)

---

In [ ]:
# Install dependencies (Unsloth, datasets, etc.)
!pip install --upgrade --no-cache-dir unsloth loth_zoo datasets huggingface_hub
# Optionally: install any other vision/model-specific dependencies if needed


## Authenticate with Hugging Face
Store your Hugging Face token in Kaggle Secrets as `HUGGINGFACE_TOKEN`. This is required to download private models/datasets.


In [ ]:
from huggingface_hub import login
import os

# Get token from Kaggle Secrets
HUGGINGFACE_TOKEN = os.environ.get('HUGGINGFACE_TOKEN', None)
if HUGGINGFACE_TOKEN is None:
    import kaggle_secrets
    HUGGINGFACE_TOKEN = kaggle_secrets.UserSecretsClient().get_secret("HUGGINGFACE_TOKEN")
login(token=HUGGINGFACE_TOKEN)


## Download and Prepare the Geo170K Dataset
We use Hugging Face Datasets to load the Geo170K dataset.


In [ ]:
from datasets import load_dataset

# Download the dataset
dataset = load_dataset('Luckyjhg/Geo170K')

# Inspect the dataset
print(dataset)
print(dataset['train'][0])


## Environment Setup for Unsloth & Model Initialization
We use Unsloth's FastVisionModel for efficient fine-tuning.
    
    **Tip:** Adjust model name and parameters as needed for your experiment.


In [ ]:
from unsloth import FastVisionModel

# Example: Qwen2-Vision model
model_name = 'Qwen/Qwen-VL-Chat'  # Replace with your preferred model
model, processor = FastVisionModel.from_pretrained(model_name)

# Optionally inspect model
print(model)


## Prepare Data for Fine-Tuning
Align the Geo170K dataset with the input format expected by your model.
    
    **Tip:** Depending on the dataset structure, you may need to adjust the preprocessing logic.


In [ ]:
def preprocess(example):
    # Example: adapt to your dataset fields
    image = example['image']
    question = example['question']
    answer = example['answer']
    # Preprocess image and text as needed
    return {
        'image': processor(image),
        'question': question,
        'answer': answer
    }

processed_dataset = dataset['train'].map(preprocess)

# Inspect processed sample
print(processed_dataset[0])


## Fine-Tuning Configuration
Set up PEFT (Parameter-Efficient Fine-Tuning) and training arguments.
    
    **Tip:** Adjust batch size, learning rate, epochs, and other parameters as needed.


In [ ]:
from unsloth import PEFTTrainer, PEFTConfig

peft_config = PEFTConfig(
    lora_r=8,
    # LoRA rank
    lora_alpha=16,
    # LoRA alpha
    lora_dropout=0.05,
    # LoRA dropout
    bias='none',
    # LoRA bias
    task_type='VISION_LANGUAGE'
    # For vision-language models
)

training_args = {
    'per_device_train_batch_size': 8,
    'num_train_epochs': 3,
    'learning_rate': 2e-4,
    'logging_steps': 50,
    'save_steps': 200,
    'output_dir': './outputs',
    'fp16': True
}


## Start Fine-Tuning
Train the model using Unsloth's PEFTTrainer.


In [ ]:
trainer = PEFTTrainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset,
    peft_config=peft_config
)

trainer.train()


## Save and Push Model
Save your fine-tuned model and optionally push to Hugging Face Hub.


In [ ]:
model.save_pretrained('./outputs/final_model')
# Optionally: push to Hugging Face
# model.push_to_hub('your-hf-username/your-model-name')


---

**Tips:**
- Adjust model, batch size, epochs, and learning rate for your specific experiment.
- Monitor GPU usage and training logs.
- Refer to [Unsloth best practices](https://www.reddit.com/r/unsloth/comments/1k12ryj/new_datasets_guide_for_finetuning_best_practices/) and [Kaggle Vision Fine-tuning Example](https://www.kaggle.com/code/danielhanchen/llama-3-2-vision-finetuning-unsloth-kaggle/notebook) for more details.

---

*Notebook generated on 2025-04-27. For support, see Unsloth and G-LLaVA documentation.*
